<a href="https://colab.research.google.com/github/christopherormerod/AI-AES-Colab/blob/main/FullExample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import datasets
import sklearn
import pandas as pd
import numpy as np

data_id = "llm-aes/asap-7-original"
data = datasets.load_dataset(data_id)
df = pd.DataFrame(data["train"])
train, test = sklearn.model_selection.train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
def aes_metrics(y1, y2):
  qwk = sklearn.metrics.cohen_kappa_score(y1, y2, weights="quadratic")
  acc = sklearn.metrics.accuracy_score(y1, y2)
  smd_numerator = np.mean(y1) - np.mean(y2)
  smd_denominator = np.sqrt((np.std(y1)**2 + np.std(y2)**2)/2)
  smd = smd_numerator / smd_denominator
  return {"QWK": qwk, "Acc": acc, "SMD":smd}

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch
from tqdm.notebook import tqdm, trange

class FineTunedEssayScorer:

  def __init__(self, model_id, max_score, min_score):
    self.max_score = max_score
    self.min_score = min_score
    self.classifier = AutoModelForSequenceClassification.from_pretrained(model_id, num_labels = max_score - min_score + 1)
    self.tokenizer = AutoTokenizer.from_pretrained(model_id)
    if torch.cuda.is_available():
      self.classifier.cuda()

  def train(self, X, y, epochs = 10):
    self.classifier.train()
    optimizer = torch.optim.AdamW(self.classifier.parameters(), lr=5e-5)
    scheduler = torch.optim.lr_scheduler.LinearLR(optimizer, start_factor=1.0, end_factor=0.0, total_iters=epochs)
    best_state = self.classifier.state_dict()
    best_score = -1
    N = len(X)
    for e in range(epochs):
      for i in tqdm(range(N)):
        optimizer.zero_grad()
        X_batch = self.tokenizer(X[i],
                                 return_tensors='pt',
                                 padding="max_length",
                                 truncation=True,
                                 max_length=512).to(self.classifier.device)
        y_batch = y[i] - self.min_score
        outputs = self.classifier(**X_batch, labels=torch.tensor([y_batch]).to(self.classifier.device))
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        sheduler.step()
      metrics = self.evalate(X, y)
      if metrics["QWK"] > best_score:
        self.best_state = self.classifier.state_dict()
        best_score = metrics["QWK"]
    self.classifier.load_state_dict(self.best_state)

  def evalate(self, X, y):
    self.classifier.eval()
    scores = self.score(X)
    return aes_metrics(y, scores)

  def score(self, X):
    self.classifier.eval()
    scores = []
    with torch.no_grad():
      for X_batch in tqdm(X):
        X_batch = self.tokenizer(X_batch,
                                 return_tensors='pt',
                                 padding="max_length",
                                 truncation=True,
                                 max_length=512).to(self.classifier.device)
        outputs = self.classifier(**X_batch)
        scores.append(int(outputs.logits.cpu().argmax(dim=1)) + self.min_score)
    return scores